In [1]:
#!pip install huggingface_hub
#pip install pypdf
#pip install langchain

## PDF

In [2]:
import numpy as np

In [3]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:\\Users\\Mahir Dursunoglu\\Desktop\\Langchain\\the-fellowship-of-the-ring.pdf")
pages = loader.load()

In [4]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [5]:
len(pages)

571

In [6]:
page = pages[501]

In [7]:
print(page.page_content[0:2500])

the mirror of galadriel 473
I am not a counsellor. You may learn something, and whether
what you see be fair or evil, that may be proﬁtable, and yet
it may not. Seeing is both good and perilous. Yet I think,Frodo, that you have courage and wisdom enough for theventure, or I would not have brought you here. Do as youwill!’
‘I will look,’ said Frodo, and he climbed on the pedestal
and bent over the dark water. At once the Mirror cleared and
he saw a twilit land. Mountains loomed dark in the distanceagainst a pale sky. A long grey road wound back out of sight.Far away a ﬁgure came slowly down the road, faint andsmall at ﬁrst, but growing larger and clearer as it approached.Suddenly Frodo realized that it reminded him of Gandalf.He almost called aloud the wizard’s name, and then he saw
that the ﬁgure was clothed not in grey but in white, in a whitethat shone faintly in the dusk; and in its hand there was awhite staff. The head was so bowed that he could see no face,and presently the ﬁgure 

In [8]:
page.metadata

{'source': 'C:\\Users\\Mahir Dursunoglu\\Desktop\\Langchain\\the-fellowship-of-the-ring.pdf',
 'page': 501}

## Document Splitting

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50, 
    length_function = len,
    separators=["\n\n", "\n","(?<=\." " ", ""]
)

In [27]:
splits =  text_splitter.split_documents(pages) 

In [28]:
len(splits)

1466

## Embeddings

In [31]:
from transformers.utils import logging
logging.set_verbosity_error()

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

## Vectorstore

In [34]:
#! pip install chromadb

In [35]:
len(splits)

1466

In [36]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "C:\\Users\\Mahir Dursunoglu\\Desktop\\Langchain\\Vectorstore"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs8"

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"},
)

documents = [splits[i].page_content for i in range(len(splits))]

collection.add(
    documents=documents,
    ids=[f"id{i}" for i in range(len(documents))]
)

In [37]:
collection.count()

1466

## Question Answering

In [54]:
context = """
I want you to answer questions related to book series named "Lord of the rings". Use the following 
passages from the first book of the serie to answer questions: {}
"""

question = """
Where Hobbits live ? Where can I find them ?
"""

related_passages = collection.query(
     query_texts=[question],
     n_results=5,
)

related_result = related_passages["documents"][0][:]
sections_str = ",".join(related_result)

In [55]:
print(sections_str)

past, and the shape of all lands has been changed; but theregions in which Hobbits then lived were doubtless the sameas those in which they still linger: the North-West of the OldWorld, east of the Sea. Of their original home the Hobbitsin Bilbo’s time preserved no knowledge. A love of learning(other than genealogical lore) was far from general amongthem, but there remained still a few in the older families whostudied their own books, and even gathered reports of oldtimes and distant lands from Elves, Dwarves, and Men. Their
own records began only after the settlement of the Shire, and,‘Happy folk are Hobbits to dwell near the shores of the,and inhabited by large families. (Bilbo and Frodo Bagginswere as bachelors very exceptional, as they were also in manyother ways, such as their friendship with the Elves.) Some-times, as in the case of the Tooks of Great Smials, or theBrandybucks of Brandy Hall, many generations of relativeslived in (comparative) peace together in one ancestral andm

In [56]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="PsiPi/liuhaotian_llava-v1.5-13b-GGUF",
  messages=[
     {"role": "system", "content": context.format(sections_str)},
     {"role": "user", "content": question},
  ],
  temperature=0.3,
)

print(completion.choices[0].message.content)


Hobbits are an ancient people, more 'a bit below the dwarves and a bit above gnomes' according to Gandalf. They primarily inhabit the North-West of the Old World, east of the Sea. Their original home is unknown, but they still linger in those regions. In the book, it is mentioned that some Hobbits lived in the Shire, which was preserved as a separate region by the Elves. The Shire is described as being 'a land of green fields and tall trees, where the grass is short and the flowers are bright, and there are no lanterns or streetlights.' It can be inferred that Hobbits live in such regions with peaceful surroundings, away from the hustle and bustle of urban life.
